In [1]:

from pyspark.sql import SparkSession
from pyspark.sql import Row
from delta import *
from pyspark.sql import functions as F
from pyspark.sql import DataFrame
from pyspark.sql.functions import when, col, monotonically_increasing_id, concat, lit, split
from pyspark.sql.functions import regexp_replace

# warehouse_location points to the default location for managed databases and tables
warehouse_location = 'hdfs://hdfs-nn:9000/warehouse'

builder = SparkSession \
    .builder \
    .master("local[2]") \
    .appName("Python Spark DataFrames and SQL") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:2.4.0") \
    .enableHiveSupport() \

spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [2]:
#----------------------------Child_Protection_Dataset-----------------------------#

In [3]:
hdfs_path_Children_Careplan = "hdfs://hdfs-nn:9000/Datasets/Bronze/Child_Protection_Dataset/Child_Protection_Dataset.csv"

In [4]:
children_careplan_df = spark.read.option("delimiter",",").option("header","true").csv(hdfs_path_Children_Careplan)

In [5]:

## Adicionar uma coluna no Dataframe com um ID artificial - nesta opereção o id irá aparecer na ultima coluna do dataframe
children_careplan_df = children_careplan_df.withColumn("Children_Careplan_Id", monotonically_increasing_id())

# list comprehension 
# utilzado para reordenar as colunas e colocar a nova coluna Childre_Careplan_Id o mais a esquerda no dataframe 
columns_ordered = ["Children_Careplan_Id"] + [col_name for col_name in children_careplan_df.columns if col_name != "Children_Careplan_Id"]
children_careplan_df = children_careplan_df.select(*columns_ordered)


In [6]:
#Verifica se a nova coluna criada existe e converte num numero inteiro
if "Children_Careplan_Id" in children_careplan_df.columns:
    children_careplan_df = children_careplan_df.withColumn("Children_Careplan_Id", col("Children_Careplan_Id").cast("int"))
else:
    print("A coluna 'Children_Careplan_Id' não existe no DataFrame.")


In [7]:

# Alterar o nome CP_End_Reason para algo mais claro e intuitivo como Careplan End Reason
children_careplan_df = children_careplan_df.withColumnRenamed("CP End Reason","Careplan End Reason")

# Alterar o nome Current CPP para algo mais claro e intuitivo como Current Careplan Protection
children_careplan_df = children_careplan_df.withColumnRenamed("Current CPP","Current Careplan Protection")

#children_careplan_df = children_careplan_df.withColumnRenamed("Old_Column_Name", "New_Column_Name")
#Alterar o nome de Reg_Disabled para algo mais claro e intuitivo como Registration Disabled
children_careplan_df = children_careplan_df.withColumnRenamed("Reg Disabled","Registration Disabled")

# Renomear e altera o simbolo & para a palavra and 
children_careplan_df = children_careplan_df.withColumnRenamed("Start Month & Year","Start Month and Year")
children_careplan_df = children_careplan_df.withColumnRenamed("End Month & Year","End Month and Year")


In [8]:

# Alteração do conteudo da coluna Category of Abuse, este quando for null passa para Category Not Available
children_careplan_df = children_careplan_df.withColumn(
   "Category of Abuse",
    when(
        col("Category of Abuse").isNull(), 
        "Category Not Available"
    ).otherwise(col("Category of Abuse"))    
)


In [9]:

# Alteração do conteudo da coluna Careplan End Reason, este quando for null passa para Criteria Not Available
children_careplan_df = children_careplan_df.withColumn(    
   "Careplan End Reason",
    when(
        col("Careplan End Reason").isNull(), 
        "Criteria Not Available"
    ).otherwise(col("Careplan End Reason"))    
)
#children_careplan_df.toPandas()

In [10]:

# Alteração do conteudo da coluna End Month & Year, este quando encontrar um valor N passa esse valor para null
children_careplan_df = children_careplan_df.withColumn(    
   "End Month and Year",
    when(
        (col("End Month and Year") == 'N'), 
        None
    ).otherwise(col("End Month and Year"))    
)


In [11]:

# Eliminação da coluna Ethnic Code, não foi possivel encontrar os significados dos valores A,B,C,D e E.
column_to_remove = "Ethnic Code"
children_careplan_df = children_careplan_df.drop(column_to_remove)


In [12]:

#criação uma nova coluna para as idades de forma a uniformizar com os outros csv para futuras possiveis analises, colocar todos em younth 0-24 years
children_careplan_df = children_careplan_df.withColumn("Uniform_Age_Group", F.expr("'Youth 0-24 years'"))


In [13]:
# criação da coluna da localização Barnet.
children_careplan_df = children_careplan_df.withColumn("Borough", F.expr("'Barnet'"))

In [14]:

#criação das colunas mes e ano correspondentes as colunas existentes

#criamos e colocamos os nomes das colunas que irão ser utilizadas numa lista 
original_columns = ["Date Careplan Started", "Date Careplan Ended", "Start Month and Year", "End Month and Year"]
#criamos uma lista vazia ara ser colocado as futuras novas colunas 
new_columns = []

# Aplicar um loop para cada coluna original 
for i, column in enumerate(original_columns, start=1):  #o enumerate é utilizado para percorrer uma sequencia (lista original_columns) e o contador começa no numero 1 e vai incrementando
    #criar os nomes das novas colunas de acordo com o numero de iteracoes 
    new_month_column = f"month{i}"
    new_year_column = f"year{i}"
    #adiciona as novas colunas na lists new_columns
    new_columns.extend([                                                
        split(col(column), " ")[0].cast("int").alias(new_month_column),     #divide a coluna original atraves do " " e converte o mes em inteiro e coloca 
        split(col(column), " ")[1].cast("int").alias(new_year_column)
    ])

# Adiciona as novas colunas ao DataFrame original
children_careplan_df = children_careplan_df.select("*", *new_columns)


# Renomear as novas colunas 
# criar duas listas com os nomes das colunas 
month_coluna = ["Data Careplan Started", "Data Careplan Ended", "Start Month and Year", "End Month and Year"]
year_coluna = ["Data Careplan Started", "Data Careplan Ended", "Start Month and Year", "End Month and Year"]

# Aplicar um loop para renomear as colunas de mês, alterando a coluna com o nome month1  para Data Careplan Started_month, o loop começa em 1 e repete para todas as colunas month
for i, prefix in enumerate(month_coluna, start=1):
    old_column_name = f"month{i}"
    new_column_name = f"{prefix}_month"
    children_careplan_df = children_careplan_df.withColumnRenamed(old_column_name, new_column_name)

# Aplicar um loop para renomear as colunas de ano, alterando a coluna com o nome year1  para Data Careplan Started_year, o loop começa em 1 e repete para todas as colunas year
for i, prefix in enumerate(year_coluna, start=1):
    old_column_name = f"year{i}"
    new_column_name = f"{prefix}_year"
    children_careplan_df = children_careplan_df.withColumnRenamed(old_column_name, new_column_name)


In [15]:

#renomear todas as colunas do dataframe onde forem encontradas " "
def rename_columns(dataframe: DataFrame) -> DataFrame:
    for column in dataframe.columns:  
        new_column_name = column.replace(" ", "_")
        dataframe = dataframe.withColumnRenamed(column, new_column_name) 
    return dataframe
    
children_careplan_df = rename_columns(children_careplan_df)

In [16]:
children_careplan_df \
    .write \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .format("delta") \
    .saveAsTable("Projeto.Child_Protection_table")

In [18]:
######### Parte Gold  ##########

In [19]:
# Ler os dados das tabelas de silver
children_careplan_df = spark.table("Projeto.Child_Protection_table")

In [20]:
# Criar a dimensão Date
dim_date_df = children_careplan_df \
              .select("Children_Careplan_Id",
                      "Data_Careplan_Started_month",
                      "Data_Careplan_Started_year",
                      "Data_Careplan_Ended_month",
                      "Data_Careplan_Ended_year",
                      "Start_Month_and_Year_month",
                      "Start_Month_and_Year_year",
                      "End_Month_and_Year_month",
                      "End_Month_and_Year_year") \
              .distinct()
                
#Criação uma coluna com chaves artificiais com valores sequenciais, colocando-a em primeiro lugar (DD = Date Dimension)
dim_date_df = dim_date_df \
                .withColumn("Date_Dimension_ID", concat(lit("DD"), monotonically_increasing_id())
                .cast("string")) \
                .select("Date_Dimension_ID", *dim_date_df.columns[0:])

In [21]:
# Criar a dimensão Child
dim_child_df = children_careplan_df \
              .select("Children_Careplan_Id",
                      "Gender",
                      "Disabled",
                      "Registration_Disabled",
                      "Unique",
                      "Current_Age_Group",
                      "Current_Age_Bracket",
                      "Age_Group_on_Registration", 
                      "Age_Group_on_deRegistration",
                      "Uniform_Age_Group") \
              .distinct()
                
#Criação uma coluna com chaves artificiais com valores sequenciais, colocando-a em primeiro lugar (CD = Child Dimension)
dim_child_df = dim_child_df \
               .withColumn("Child_Dimension_ID", concat(lit("CD"), monotonically_increasing_id())
               .cast("string")) \
               .select("Child_Dimension_ID", *dim_child_df.columns[0:])

In [22]:
# Criar a dimensão Care Plan
dim_care_plan_df = children_careplan_df \
                   .select("Children_Careplan_Id",
                           "Category_of_Abuse",
                           "Careplan_End_Reason",
                           "Current_Careplan_Protection",
                           "Borough") \
                   .distinct()
                
#Criação uma coluna com chaves artificiais com valores sequenciais, colocando-a em primeiro lugar (CPD = Care Plan Dimension)
dim_care_plan_df = dim_care_plan_df \
                  .withColumn("Care_Plan_Dimension_ID", concat(lit("CPD"), monotonically_increasing_id())
                  .cast("string")) \
                  .select("Care_Plan_Dimension_ID", *dim_care_plan_df.columns[0:])

In [23]:
# Criação da tabela de factos Care Plan per Child

# Juntar o ID da dimensão Date e Child
cp_dc_df = dim_date_df \
          .join(dim_child_df, dim_date_df.Children_Careplan_Id == dim_child_df.Children_Careplan_Id) \
          .select("Date_Dimension_ID", "Child_Dimension_ID", dim_date_df.Children_Careplan_Id.alias("Children_Careplan_Id"))

# Juntar o ID da dimensão Care Plan
cp_per_child_df = cp_dc_df \
                  .join(dim_care_plan_df, cp_dc_df.Children_Careplan_Id == dim_care_plan_df.Children_Careplan_Id) \
                  .select("Date_Dimension_ID", "Child_Dimension_ID", "Care_Plan_Dimension_ID")

# Criação de uma coluna com valor 1 em todas as linhas que simboliza que a criança esteve num Care Plan
cp_per_child_df = cp_per_child_df.withColumn("Child_with_CP", lit(1))

In [24]:
# Remover colunas desnecessárias
dim_date_df.drop("Children_Careplan_Id")
dim_child_df.drop("Children_Careplan_Id")
dim_care_plan_df.drop("Children_Careplan_Id")

# Escrever para a tabela delta
dim_date_df.write \
           .format("delta") \
           .mode("overwrite") \
           .option("overwriteSchema", "true") \
           .save("hdfs://hdfs-nn:9000/warehouse/Projeto.db/dim_date")

#dim_data_df.toPandas()

dim_child_df.write \
            .format("delta") \
            .mode("overwrite") \
            .option("overwriteSchema", "true") \
            .save("hdfs://hdfs-nn:9000/warehouse/Projeto.db/dim_child")

#dim_child_df.toPandas()

dim_care_plan_df.write \
                .format("delta") \
                .mode("overwrite") \
                .option("overwriteSchema", "true") \
                .save("hdfs://hdfs-nn:9000/warehouse/Projeto.db/dim_care_plan")

#dim_care_plan_df.toPandas()

cp_per_child_df.write \
               .format("delta") \
               .mode("overwrite") \
               .option("overwriteSchema", "true") \
               .save("hdfs://hdfs-nn:9000/warehouse/Projeto.db/cp_per_child")

In [25]:
spark.stop()